In [ ]:
%pip install -r PredictionModel/requirements.txt

In [1]:
#!python PredictionModel/runPredModel.py patient-data/p001/p001.jpg
from PredictionModel.runPredModel import scanallxrays
import os
import pprint

pdir = "patient-data"
for p in os.listdir(pdir):
    patient_dir = os.path.join(pdir, p)
    print(f"patient {p}: {patient_dir}")
    pprint.pprint(scanallxrays(patient_dir))

patient p001: patient-data\p001
{'Atelectasis': 0.21498525887727737,
 'Cardiomegaly': 0.14202095940709114,
 'Pleural_Thickening': 0.3676226958632469,
 'Pneumonia': 0.43097905814647675}
patient p002: patient-data\p002
{'Atelectasis': 0.17979608476161957,
 'Cardiomegaly': 0.17245332896709442,
 'Pleural_Thickening': 0.3351752310991287,
 'Pneumonia': 0.4001656621694565}
patient p003: patient-data\p003
{'Atelectasis': 0.4432849735021591,
 'Cardiomegaly': 0.3742854967713356,
 'Pleural_Thickening': 0.32515817135572433,
 'Pneumonia': 0.5054616332054138}
patient p004: patient-data\p004
{'Atelectasis': 0.24015431,
 'Cardiomegaly': 0.071538076,
 'Pleural_Thickening': 0.20353583,
 'Pneumonia': 0.27674294}
patient p005: patient-data\p005
{'Atelectasis': 0.6679766873518626,
 'Cardiomegaly': 0.5434231907129288,
 'Pleural_Thickening': 0.3726285013059775,
 'Pneumonia': 0.8030462066332499}
patient p006: patient-data\p006
{'Atelectasis': 0.46802014112472534,
 'Cardiomegaly': 0.505253255367279,
 'Pleural_

In [ ]:
from PredictionModel.convertDcm import convertall

convertall("patient-data/p001")

In [ ]:
from PredictionModel.heatmaps import genheatmap
from PredictionModel.runPredModel import diseases


img_path = "patient-data/p001/p001.dcm"
# pathology = 'Atelectasis'

for ds in diseases:
    print(f"Disease: {ds}")
    genheatmap(img_path, ds)
    break

In [ ]:
from PredictionModel.heatmaps import genheatmappatient

patient_path = "patient-data/p001"
genheatmappatient(patient_path)